In [1]:
# Import dependencies
import train_xgb
import json

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-154990
aml-quickstarts-154990
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=1,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:

from azureml.data.dataset_factory import TabularDatasetFactory
# Try to load the dataset from the workspace. Otherwise, load if from Kaggle
found = False
ds_key = 'Ames-housing-dataset'
ds_desc = 'Ames Housing training data.'

if ds_key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[ds_key]
    print(f'Found registered {ds_key}, use it.')
    
if not found:
    train, test = train_xgb.load_data_clean(source='kaggle')
    print(f"train.shape = {train.shape}, test.shape = {test.shape}")
    # Register the train dataset
    blob = ws.get_default_datastore()
    dataset = TabularDatasetFactory.register_pandas_dataframe(train, blob, name=ds_key, description=ds_desc)

train.shape = (1460, 80), test.shape = (1459, 79)


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/c0e9801f-3f05-4219-9718-9b8b650ee129/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


# XGBoostRegressor Model Deployment

In [5]:
%%writefile conda_env.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults==1.32.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
channels:
- anaconda
- conda-forge

Writing conda_env.yml


In [6]:
with open('hdr-outputs/best_hdr_metrics.json', 'r') as file:
    best_hdr_metrics = json.load(file)

In [7]:
from azureml.core import Model
# Register the best model
model = Model.register(ws, model_path='hdr-outputs/model.pkl', model_name='Ames-Housing-XGB-Model', tags=best_hdr_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-XGB-Model
Ames-Housing-XGB-Model	Ames-Housing-XGB-Model:1	1


In [8]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "XGB"},
                                              description="Predict Ames Housing Prices",
                                              auth_enabled=True,
                                              enable_app_insights=True)

In [9]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('conda_env.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [10]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-XGB-Model')

myenv = Environment.get(workspace=ws, name="project-env")

inference_config = InferenceConfig(entry_script="entry_script.py", environment=myenv)

service_name = 'ames-housing-xgb-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-19 05:38:48+00:00 Creating Container Registry if not exists..
2021-08-19 05:48:48+00:00 Registering the environment..
2021-08-19 05:48:51+00:00 Building image..
2021-08-19 05:55:52+00:00 Generating deployment configuration..
2021-08-19 05:55:53+00:00 Submitting deployment to compute.
2021-08-19 05:55:56+00:00 Checking the status of deployment ames-housing-xgb-4006..
2021-08-19 05:58:40+00:00 Checking the status of inference endpoint ames-housing-xgb-4006.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 2.55 s, sys: 295 ms, total: 2.85 s
Wall time: 19min 59s


In [11]:
print(service.get_logs())

2021-08-19T05:58:30,138589800+00:00 - rsyslog/run 
2021-08-19T05:58:30,140069900+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-08-19T05:58:30,164580700+00:00 - iot-server/run 
2021-08-19T05:58:30,186601400+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-19T05:58:30,508799700+00:00 - iot-server/finish 1 0
2021-08-19T05:58:30,511827400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (61)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 89
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-19 05:58:32,105 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-19 05:58:32,110 | root | INFO | Starting up request id generator
2021-08-19 05:58:32,110 | root | INFO | Starting up app in

In [12]:
# Prepare data for request
_ , test = train_xgb.load_data_clean()
test = train_xgb.label_encode(test)
data = {'data': test.head().to_dict(orient='list')}

# Replace the next cell with the code from 'Consume' tab of the endpoint
# and delete 'data = {}' assignment as data is defined in this cell!

In [13]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# data = {
# }

body = str.encode(json.dumps(data))

url = 'http://1e0b5126-672b-4e49-8aa7-7ae581c2aa03.southcentralus.azurecontainer.io/score'
api_key = 'Vvo2X6KqfTk8WdCmam8rscslkmY5NsCd' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [124214.375, 159119.453125, 175328.28125, 186762.34375, 190019.46875]}"'


In [14]:
print(service.get_logs())
# Clean up resources
service.delete()

2021-08-19T05:58:30,138589800+00:00 - rsyslog/run 
2021-08-19T05:58:30,140069900+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-08-19T05:58:30,164580700+00:00 - iot-server/run 
2021-08-19T05:58:30,186601400+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-19T05:58:30,508799700+00:00 - iot-server/finish 1 0
2021-08-19T05:58:30,511827400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (61)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 89
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-19 05:58:32,105 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-19 05:58:32,110 | root | INFO | Starting up request id generator
2021-08-19 05:58:32,110 | root | INFO | Starting up app in

# Automated ML Model Deployment

In [15]:
with open('aml-outputs/best_aml_metrics.json', 'r') as file:
    best_aml_metrics = json.load(file)

In [16]:
# Register the best model
model = Model.register(ws, model_path='aml-outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_aml_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:1	1


In [17]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "AutoML"},
                                              description="Predict Ames Housing Prices",
                                              auth_enabled=True,
                                              enable_app_insights=True)

In [18]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('aml-outputs/conda_env_v_1_0_0.yml')
env.python.conda_dependencies = cd
# Creates the environment inside a Docker container.
# env.docker.enabled = True
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [19]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-AutoML-Model')

myenv = Environment.get(workspace=ws, name="project-env")
inference_config = InferenceConfig(entry_script="aml-outputs/scoring_file_v_1_0_0.py", environment=myenv)

service_name = 'ames-housing-aml-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-19 06:03:11+00:00 Creating Container Registry if not exists.
2021-08-19 06:03:11+00:00 Registering the environment.
2021-08-19 06:03:13+00:00 Building image..
2021-08-19 06:18:48+00:00 Generating deployment configuration..
2021-08-19 06:18:49+00:00 Submitting deployment to compute.
2021-08-19 06:18:52+00:00 Checking the status of deployment ames-housing-aml-0812..
2021-08-19 06:22:48+00:00 Checking the status of inference endpoint ames-housing-aml-0812.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e29418d6-3866-4603-91f8-0b2cb756ebc1
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: ames-housing-aml-0812. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image c726cd197c8d4e64902174bc769dc498.azurecr.io/azureml/azureml_e434aed78f9426216c7e0e89c0851446 locally. Please refer to https://aka.ms/debugimage#service-lau

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e29418d6-3866-4603-91f8-0b2cb756ebc1
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: ames-housing-aml-0812. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image c726cd197c8d4e64902174bc769dc498.azurecr.io/azureml/azureml_e434aed78f9426216c7e0e89c0851446 locally. P

In [20]:
print(ws.webservices['ames-housing-aml-0812'].get_logs())

None


In [ ]:
# Prepare data for request
_ , test = train_xgb.load_data_clean()
data = {'data': test.head().to_dict(orient='list')}

# Replace the next cell with the code from 'Consume' tab of the endpoint
# and delete 'data = {}' assignment as data is defined in this cell!

In [21]:
print(service.get_logs())
# Clean up resources
service.delete()
cpu_cluster.delete()

2021-08-19T06:26:27,601881200+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-08-19T06:26:27,606814100+00:00 - rsyslog/run 
2021-08-19T06:26:27,614454100+00:00 - iot-server/run 
2021-08-19T06:26:27,700982400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_59fba923920e15fa8fbe872c700ad33f/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-19T06:26:28,065364700+00:00 - iot-server/finish 1 0
2021-08-19T06:26:28,071959800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (544)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 572
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Exception in worker process
Traceback (most recent call last):
  File "/var/azureml-server/routes_common.py", line 37, in <